In [ ]:
import requests
import random
import json
from hashlib import md5
import os
import json
import time


def translate_text(query):
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    # Set your own appid/appkey.
    appid = 'APP_ID'
    appkey = 'APP_KEY'

    from_lang = 'en'
    to_lang =  'zh'

    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()

    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)

    headers = {"Content-Type": 'application/x-www-form-urlencoded'}
    payload = {"appid": appid, "q": query, "from": from_lang, "to": to_lang, "salt": salt, "sign": sign}

    r = requests.post(url, params=payload, headers=headers)
    result = r.json()

    return result

def translate_file(folder_path, filename, output_path):
    file_path = os.path.join(folder_path, filename)
    # 打开JSON文件并逐行读取并打印内容
    translate_source = [] 
    results = []
    with open(file_path, 'r') as file:
      qapair = json.load(file)
      # 50句question一组，每句话\n隔开。每次翻译一组

    for i in range(0, len(qapair), 50):
        query = '\n'.join([item['question'] for item in qapair[i:i+50]])
        translate_source.append(query)

    requests_per_second = 5  # 每秒请求的次数
    for query in translate_source:
        result = translate_text(query)
        results.append(result)  
        time.sleep(1 / requests_per_second)

    rearangeed_results = []
    for result in results:
        for item in result['trans_result']:
            rearangeed_results.append(item)

    translateed_file = []
    for json_item in qapair:
        for chinese in rearangeed_results:
            if json_item['question'] == chinese['src']:
                json_item['chinese question'] = chinese['dst']
                translateed_file.append(json_item)
                break
    
    with open(os.path.join(output_path, filename), 'w', encoding="utf-8") as file:
        json.dump(translateed_file, file, indent=4, ensure_ascii=False)


folder_path = './results/3_21/json/'
output_path = './results/3_21/ch_baidu/'

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        translate_file(folder_path, filename, output_path)
        print(f'{filename} has been translated')